<a href="https://colab.research.google.com/github/tianxiang84/PINN-elastodynamics/blob/wave_1d_rod/ElasticWaveRod/PINN_Rod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import libraries

In [1]:
# Platform-related functions
import platform
# System specific functions
import sys
# Dealing with I/O streams
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
# Time-related functions
import time
# File operation
import shutil
# Serializing and de-serializing a Python object structure
import pickle

# Setup GPU for training
%tensorflow_version 1.x
import tensorflow as tf
tf.set_random_seed(1111)
# Math functions defined in C standard
import math
# Multi-dimensional array calculation
import numpy as np
np.random.seed(1111)
# Data analysis and manipulation
import pandas as pd
# Dealing with MATLAB files
import scipy.io
# Desig n of experiments for Python
!pip install --upgrade pyDOE
from pyDOE import lhs

# For plotting
import matplotlib
if platform.system()=='Linux':
    matplotlib.use('Agg')
if platform.system()=='Windows':
    from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

TensorFlow 1.x selected.
Requirement already up-to-date: pyDOE in /usr/local/lib/python3.6/dist-packages (0.3.8)


Check GPU availability (On Google Colab, please use Runtime --> Change runtime type is you see an empty list as output.)

In [2]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

['/device:GPU:0']